#  Enrutamiento con LLM Router Chain

## Importar librerías iniciales e instancia de modelo de chat

In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SequentialChain, LLMChain #importamos el SequentialChain que es el modelo completo
f = open('/home/iabd/clave_gpt.txt')
api_key = f.read().strip()
llm = ChatOpenAI(openai_api_key=api_key)

### Plantillas de enrutamiento

In [2]:
#Template (prompt) para soporte básico a clientes de coches
plantilla_soporte_basico_cliente = '''Eres una persona que asiste a los clientes de automóviles con preguntas básicas que pueden
necesitar en su día a día y que explica los conceptos de una manera que sea simple de entender. Asume que no tienen conocimiento
previo. Esta es la pregunta del usuario/n{input}'''

In [3]:
#Template (prompt) para soporte avanzados a nuestros expertos en mecánica
plantilla_soporte_avanzado_mecánico = '''Eres un experto en mecánica que explicas consultas avanzadas a los mecánicos
de la plantilla. Puedes asumir que cualquier que está preguntando tiene conocimientos avanzados de mecánica. 
Esta es la pregunta del usuario/n{input}'''

In [4]:
#Se pueden añadir todas las plantillas que necesitemos

### Prompts a enrutar

In [5]:
#Debemos crear una lista de diccionarios, cada diccionario contiene su nombre, la descripción (en base a la cual el enrutador
#hará su trabajo) y el prompt a usar en cada caso
prompt_infos = [
    {'name':'mecánica básica','description': 'Responde preguntas básicas de mecánicas a clientes',
     'prompt_template':plantilla_soporte_basico_cliente},
    {'name':'mecánica avanzada','description': 'Responde preguntas avanzadas de mecánica a expertos con conocimiento previo',
     'prompt_template':plantilla_soporte_avanzado_mecánico},
    
]

In [6]:
#TODO LO QUE VIENE A CONTINUACIÓN ES AGNÓSTICO DEL CASO DE USO Y LO PUEDES UTILIZAR PARA TODOS LOS CASOS DE USO

### ConversationChain

In [7]:
from langchain.chains import LLMChain

In [8]:
#Creamos un diccionario de objetos LLMChain con las posibles cadenas destino
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

/tmp/ipykernel_5691/3171874237.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [9]:
destination_chains

{'mecánica básica': LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='Eres una persona que asiste a los clientes de automóviles con preguntas básicas que pueden\nnecesitar en su día a día y que explica los conceptos de una manera que sea simple de entender. Asume que no tienen conocimiento\nprevio. Esta es la pregunta del usuario/n{input}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x781ab3956f90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x781ab39ec9b0>, root_client=<openai.OpenAI object at 0x781abc3462d0>, root_async_client=<openai.AsyncOpenAI object at 0x781ab3956fc0>, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}),
 'mecánica avanzad

In [10]:
#Creamos el prompt y cadena por defecto puesto que son argumento obligatorios que usaremos posteriormente
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

### Multi Routing Template

In [11]:
#Importamos una plantilla que podremos formatear su parámetro {destinations} que tendrá cada nombre y descripción de la información de prompts
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [12]:
print(MULTI_PROMPT_ROUTER_TEMPLATE) #El parámetro importante es {destinations}, debemos formatearlo en tipo string

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

### Destinos de Routing

In [13]:
#Creamos una string global con todos los destinos de routing usando el nombre y descripción de "prompt_infos"
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [14]:
destinations_str

'mecánica básica: Responde preguntas básicas de mecánicas a clientes\nmecánica avanzada: Responde preguntas avanzadas de mecánica a expertos con conocimiento previo'

### Router Prompt

In [15]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [16]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str #Formateamos la plantilla con nuestros destinos en la string destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), #Para transformar el objeto JSON parseándolo a una string
)

In [17]:
print(router_template) #verificamos que se ha formateado correctamente

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
mecánica básica: Responde preguntas básicas de mecánicas a clientes
mecánica avanzada: Responde pregu

### Routing Chain Call

In [18]:
from langchain.chains.router import MultiPromptChain

In [19]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [20]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, #El objeto con los posibles LLMChain que creamos al inicio
                         default_chain=default_chain, verbose=True #Indicamos el LLMChain por defecto (obligatorio)
                        )

/tmp/ipykernel_5691/2474013140.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [21]:
chain.invoke("¿Cómo cambio el aceite de mi coche?")



> Entering new MultiPromptChain chain...
mecánica básica: {'input': '¿Cómo cambio el aceite de mi coche?'}
> Finished chain.


{'input': '¿Cómo cambio el aceite de mi coche?',
 'text': 'Cambiar el aceite de tu coche es una tarea importante para mantener el motor en buenas condiciones. Aquí te dejo algunos pasos sencillos para que puedas hacerlo tú mismo:\n\n1. Reúne todos los materiales necesarios: aceite de motor nuevo, filtro de aceite nuevo, recipiente para recoger el aceite usado, llave de filtro de aceite, llave de drenaje de aceite, embudo y trapos.\n\n2. Estaciona tu coche en una superficie plana y deja enfriar el motor por unos minutos.\n\n3. Localiza el tapón de drenaje de aceite en el fondo del motor y coloca el recipiente debajo. Afloja el tapón con la llave de drenaje y deja que el aceite usado drene completamente.\n\n4. Retira el filtro de aceite antiguo con la llave correspondiente y coloca el nuevo filtro en su lugar, asegurándote de lubricar el anillo de sellado con un poco de aceite limpio.\n\n5. Vuelve a colocar el tapón de drenaje de aceite y apriétalo correctamente. Llena el motor con la ca

In [22]:
chain.invoke("¿Cómo funciona internamente un catalizador?")



> Entering new MultiPromptChain chain...
mecánica avanzada: {'input': '¿Cómo funcionan internamente los catalizadores?'}
> Finished chain.


{'input': '¿Cómo funcionan internamente los catalizadores?',
 'text': 'Los catalizadores son dispositivos muy importantes en el sistema de escape de un vehículo, ya que tienen la función de reducir las emisiones contaminantes producidas en la combustión del motor. \n\nInternamente, un catalizador está formado por un sustrato cerámico recubierto con metales preciosos como platino, paladio y rodio, que actúan como catalizadores en las reacciones químicas que tienen lugar dentro de él. \n\nCuando los gases de escape pasan a través del catalizador, los hidrocarburos no quemados, el monóxido de carbono y los óxidos de nitrógeno se encuentran con los metales preciosos en la superficie del sustrato cerámico. Estos metales actúan como catalizadores y facilitan la reacción química que convierte estos gases nocivos en gases menos dañinos como dióxido de carbono, agua y nitrógeno. \n\nEl diseño del sustrato cerámico del catalizador permite una mayor superficie de contacto entre los gases de escap